In [ ]:
import os

from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma


from dotenv import load_dotenv

load_dotenv()

In [ ]:
# import torch

loader = PyPDFLoader(os.environ["DOCUMENT_PATH"])
documents = loader.load_and_split()

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(documents, embeddings)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# embeddings = HuggingFaceInstructEmbeddings(
#     query_instruction="Represent the query for retrieval: "
# )
# embedded_docs = embeddings.embed_documents(documents)
# docsearch = Chroma.from_embedded_documents(embedded_docs)


In [ ]:
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.0), 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 50}),
                                 return_source_documents=True)

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Du bist ein erfahener Shadowrun Spielleiter und beantwortest Fragen zu den Regeln wahrheitsgemäß. Du spielst die 5. Edition der Regeln. Wenn du etwas nicht weißt, antworte entsprechend.{query}")

query = "Kann ich mit einer Nahkampf- und Fernkampfwaffe gleichzeitig angreifen, wenn der Charakter den Vorteil Beidhändig hat?"
query = prompt_template.format(query=query)
results = qa(query)
results

In [ ]:
query = "Wie funktioniert Rückstoß?"

query = prompt_template.format(query=query)
results = qa(query)
results